# Phyloseq and Microbiome analysis in R

There are a number of packages developed in R that make microbiome analysis easy and produce great figures.
There is at times a lot of overlap between this analysis and QIIME - and the choice is yours.

Personally I think that R allows alot more freedom and is more aesthetically pleasing, plus there are lots of things that cannot be done in QIIME that can be done in R.
Having said that it does require some knowledge of R, and however this notebook tries to step you through what you need to know.

Two packages in particular are useful for microbiome analysis - the microbiome package builds on phyloseq, and you may find you don't need the microbiome package.

[Phyloseq](http://joey711.github.io/phyloseq/)

[Microbiome](http://microbiome.github.io/microbiome/)

As always there is more than one way to do things, this phyloseq import will follow the basic import tutorial, in other words importing files individually to create a phyloseq object. However, you may like to follow their tutorials on the QIIME, or mothur input. I do find that as these programs change over time there can be a delay in updating the latest data import workflow, hence why I have gone with the metheod of phyloseq-ize Data already in R that will work no matter what updates are made to QIIME, usearch or other pipelines you may use.

** Libraries**

*Note: You will need to install the library first if you haven't already using* `install.packages`

Load the libraries

    library("phyloseq")
    library("tidyverse")
    library("ape")
    library("plyr")
    library("readr")
    library("microbiome")
    library("lattice")
    library("colorspace")
    library("RColorBrewer")
    library("vegan")
    library("microbiome")
    library("jsonlite")
    library("xlsx")
    
Remove variables in current environment

    rm(list=ls())

Set working directory

    setwd("~/tick_meta_analysis/data")
    
*_Remember the useful package for data management in R_ [ProjectTemplate](http://projecttemplate.net/getting_started.html)

## Importing data and creating a physeq object

Next you can set your working directory and import your OTU table (from the usearch pipeline) and your taxonomy table (from QIIME). I recommend you copy and data the files from your Usearch/QIIME directory output to the data file in the R management directory
    
    setwd("~/tick_meta_analysis/data")
    
Import OTU table taking note of the following:
- Rows = taxa (otus) and columns = samples. The sample names should already be matched to your metadata if you following the QIIME analysis to generate a feature table correctly.
- Order the OTU column in ascending order _to do this you may need to delete the letters 'OTU', which can be done in excel with a `control-H` command._
- The first column with the OTU number must then be deleted, this can get confusing so it is best to take this document as a copy so that you do not save over the table with the OTU numbers.
- Ensure that the column headings (sample names) are in the same order as what appears in the metadata sheet. In excel you may like to copy and paste the data using the transpose option to order the sample column before reverting it back.
- Save as `csv` file

```
uparse_otus <- read_csv("~/tick_meta_analysis/data/otu_table.csv")
otumat <- as.matrix(uparse_otus)
rownames(otumat) <- paste0("OTU", 1:nrow(otumat))
colnames(otumat) <- paste0("Sample", 1:ncol(otumat))
otumat
```

Import the taxonomy table taking note of the following:
- Rows = taxa and columns = taxa heirachy
- The first column with the OTU number must then be deleted, this can get confusing so it is best to take this document as a copy so that you do not save over the table with the OTU numbers.
- Save as `csv` file

```
colnames(taxmat) <- c("Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species")
taxonomy <- read_csv("~/tick_meta_analysis/data/taxonomy.csv")
taxmat <- as.matrix(taxonomy)
rownames(taxmat) <- rownames(otumat)
colnames(taxmat) <- c("Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species")
taxmat
```

Next wcan check the class of the `otumat` and `taxmat` objects, they **MUST** be in matrix format. Then we can great a phyloseq object called `physeq` from the otu and taxonomy tables and check the sample names.
```
class(otumat)
class(taxmat)
OTU = otu_table(otumat, taxa_are_rows = TRUE)
TAX = tax_table(taxmat)
OTU
TAX
physeq = phyloseq(OTU, TAX)
physeq
sample_names(physeq)
```

**There are a few options for the tree import**- you can create a random tree using the `ape` package, however as this only using taxonomy information it is not as robust as other methods that use the sequence information, such as that generated by QIIME or USEARCH.

```
random_tree = rtree(ntaxa(physeq), rooted=TRUE, tip.label=taxa_names(physeq))
plot(random_tree)
```

To import your usearch tree
```
MyTree <- read.tree("otus.tree")
```
To import your QIIME2 tree
```
MyTree2 <- read.tree("rooted-tree.nwk")
```

Now you can merge your data to create a final phyloseq object
```
physeq1 = merge_phyloseq(physeq, sampledata, random_tree)
physeq1
```

You should end up with the following output after `physeq1`

>```
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 15985 taxa and 1390 samples ]
sample_data() Sample Data:       [ 1390 samples by 13 sample variables ]
tax_table()   Taxonomy Table:    [ 15985 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 15985 tips and 15984 internal nodes ]
>```

***

## Basic commands to check your physeq object
```
nsamples(physeq1)
ntaxa(physeq1)
sample_names(physeq1)[1:5]
rank_names(physeq1)
sample_variables(physeq1)
otu_table(physeq1)[1:5, 1:5]
tax_table(physeq1)[1:5, 1:4]
myTaxa = names(sort(taxa_sums(physeq1), decreasing = TRUE)[1:10])
```


## Pre-processing data
The phyloseq package also includes functions for filtering, subsetting, and merging abundance data. Filtering in phyloseq is designed in a modular fashion similar to the approach in the genefilter package. This includes the `prune_taxa` and  `prune_samples` methods for directly removing unwanted indices, as well as the `filterfun_sample` and `genefilter_sample` functions for building arbitrarily complex sample-wise filtering criteria, and the `filter_taxa` function for taxa-wise filtering. 

##### Transforming and filtering data
In the following example, the Physeq1 data is first transformed to relative abundance, creating the new PHr object, which is then filtered such that only OTUs with a mean greater than 10^-5 are kept.
```
PHr  = transform_sample_counts(physeq1, function(x) x / sum(x) )
PHfr = filter_taxa(PHr, function(x) mean(x) > 1e-5, TRUE)
```
Remeber you can check the number of taxa at any stage by
```
ntaxa(PHr)
ntaxa(PHfr)
```
This results in a highly-subsetted object, `PHfr`, containing just 4624 of the original ~19216 OTUs. Note that in both lines we have provided a custom function for transformation and filtering, respectively.

##### Subsetting data
Subset import taxa, and set threshold limites, e.g. >20 reads.

```
c_Alphaprob = subset_taxa(physeq1, Class=="c__Alphaproteobacteria")
c_Alphaprob = prune_samples(sample_sums(c__Alphaprob)>=20, c__Alphaprob)

g_Borrelia = subset_taxa(physeq1, Genus=="Borrelia")
g_Borrelia = prune_samples(sample_sums(g_Borrelia)>=1, g_Borrelia)

f_Bartonellaceae = subset_taxa(physeq1, Family=="Bartonellaceae")
f_Bartonellaceae = prune_samples(sample_sums(f_Bartonellaceae)>=1, f_Bartonellaceae)
```

#### Plotting subsets
```
title = "plot_bar; Alphaproteobacteria-only"
plot_bar(c__Alphaprob, "Genus_species", title=title)
```

## Rarefying data

<div class="alert alert-warning"> Please note that the authors of phyloseq do not advocate using this as a normalization procedure, despite its recent popularity. See McMurdie and Holmes 2015 PLoS Comput. Biol. or "?rarefy_even_depth" in your R environment </div>

You will need to experiment with this line of code in order for it to make biological sense to your data set. We recommend reading the reference above about Rarefying microbial datasets.

```
physeq1.rarified <- rarefy_even_depth(physeq1)
ntaxa(physeq1.rarified)
nsamples(physeq1.rarified)
rm(physeq1.rarified)
physeq1.rarified <- rarefy_even_depth(physeq1, rngseed = 1000)
ntaxa(physeq1.rarified)
nsamples(physeq1.rarified)

```